# Generazione di famiglie di porte e finestre attraverso l'analisi di una matrice booleana (occupazione degli spazi)

### Modelli scelti per la riproduzione:

## Single modern door ("firstDoor")
<img src="files/firstDoor.jpg" width="200">

#### Link: https://s-media-cache-ak0.pinimg.com/originals/b1/b5/33/b1b53324b4b5c35bbf6b93d643a88ee9.jpg

## Black modern door ("secondDoor")
<img src="files/secondDoor.jpg" width="250">

#### Link: http://nebula.wsimg.com/b728afb8df0e66870253fd95e5418142?AccessKeyId=5584AEB52B71BD158E2C&disposition=0&alloworigin=1

## Style modern window ("firstWindow")
<img src="files/firstWindow.jpg" width="500">

#### Link: https://www.andersenwindows.com/-/media/andersenwindows/images/home-style-library/quintessential-windows/prairie-module-2-qw.jpg?h=354&w=514&la=en&hash=CB0723270341483E562C230B9F45C4CBCCDEE845

## Black modern window ("secondWindow")
<img src="files/secondWindow.jpg" width="250">

#### Link: https://cloud.graphicleftovers.com/10316/item12097/12097_watermark.jpg





Importo la libreria pyplasm

In [35]:
from pyplasm import *

Definisco una funzione che, data una lista di reali, mi restituisce la somma dei suoi elementi. Mi servirà per confrontare la lista delle misure delle celle con quella totale imposta per l'intero solido.

In [36]:
def sumAllElements(myList):
    """
    Funzione per sommare gli elementi di una lista.
    
    Args:
        myList (lista di reali): lista di cui calcolare la somma degli elementi
        
    Returns:
        s (numero reale): è la soma degli elementi della lista 
    """
    s = 0
    for val in myList:
        s += val
    return s


Definisco una funzione per adattare gli elementi di una lista ad un valore che ne rappresenta la somma. Mi servirà per adattare le misure delle celle a quella totale imposta per l'intero solido.

In [37]:
def adaptListToDimension(myList,dim):
    """
    Funzione per adattare gli elementi di una lista ad un valore dello stesso tipo: 
    se il valore dato differisce dalla somma degli elementi della lista, questi ultimi vengono modificati. 
    
    Args:
        myList (lista di reali): lista di numeri da adattare
        dim (reale o intero): valore di riferimento su cui riadattare gli elementi della lista
        
    Returns:
        myList (lista di reali): è la lista opportunatamente riadattata (la somma degli elementi è ora uguale a "dim") 
    """
    summatory = sumAllElements(myList)
    diff = summatory - dim
    diff = diff / len(myList)
    correctList = []
    if summatory is not dim:
        for val in myList : correctList.append(val - diff)
        return correctList
    return myList


Definisco una funzione per generare famiglie di porte o finestre in 2D (sul piano YZ). Si tratta di una funzione del secondo ordine che utilizza 2 gruppi di parametri. 
- Il primo gruppo è costituito dalle liste delle dimensioni delle celle che compongono il solido rispettivamente sull' asse Y e sull' asse Z. Inoltre fa parte di questo primo gruppo una matrice bidimensionale che rappresenta l'occupazione del solido nelle rispettive celle (ciascun elemento della matrice rappresenta una cella [in questo caso piana] del solido; il valore 0 simboleggia che il solido è "vuoto" nella cella corrispondente; il valore 1 simboleggia che il solido è "vuoto" nella cella corrispondente)
- Il secondo gruppo di parametri è costituito dalle dimensioni totali che deve occupare il solido. Se le dimensioni delle celle non sono conocrdi con quelle massime del solido le prime verranno riadattate.

Successivamente testo la funzione

In [38]:
def door2D (Ylist,Zlist,occupancy):
    """
    Funzione per generare famiglie di porte o finestre in 2D (piano YZ)
    
    Args:
        Ylist (lista di reali): lista delle dimensioni sull'asse Y di tutte le celle che compongono il solido
        Zlist (lista di reali): lista delle dimensioni sull'asse Y di tutte le celle che compongono il solido
        occupancy (matrice bidimensionale): matrice che rappresenta l'occupazione o meno di una cella da parte del "solido pieno"
        
    Returns:
        door2D0 (funzione): funzione annidata
    """
    def door2D0(dy,dz):
        """
        Args:
            dy (numero reale): dimensione massima su y del solido
            dz (numero reale): dimensione massima su z del solido
        
        Returns:
            structure (oggetto hpc): porta o finestra riprodotta
        """
    
        Y = adaptListToDimension(Ylist,dy)
        Z = adaptListToDimension(Zlist,dz)
        
        occ = occupancy
        
        structure = STRUCT([CUBOID([0,0,0])])

        sumZ = 0
        i  = 0
        
        for row in occ :
            tmpY = []
            j=0
            for val in row:
                
                if val: tmpY.append(Y[j])
                else: tmpY.append(-Y[j])
                j+=1
                
            qy = QUOTE(tmpY)
            qz = QUOTE([Z[i]])
            pxy = PROD([QUOTE([0]),qy])
            pyz = PROD([pxy,qz])
        
            structure = STRUCT([structure,T([3])([sumZ]),pyz])
            sumZ += Z[i]
            i += 1
            
        return structure
    return door2D0

#VIEW(door2D([.3,.3,.3,.1],[.1,.3,.4,.5,.2],[[1,1,1,1],[1,0,0,1],[1,1,1,1],[1,0,0,1],[1,1,1,1]])(10.0,15.0))

Definisco una funzione per generare famiglie di porte o finestre in 3D (nello spazio XYZ). Si tratta di una funzione del secondo ordine che utilizza 2 gruppi di parametri.(Ho esteso la funzione precedente anche alla terza dimensione)
- Il primo gruppo è costituito dalle liste delle dimensioni delle celle che compongono il solido rispettivamente sugli assi X, Y e Z. Inoltre fa parte di questo primo gruppo una matrice tridimensionale che rappresenta l'occupazione del solido nelle rispettive celle (ciascun elemento della matrice rappresenta una cella del solido; il valore 0 simboleggia che il solido è "vuoto" nella cella corrispondente; il valore 1 simboleggia che il solido è "vuoto" nella cella corrispondente)
- Il secondo gruppo di parametri è costituito dalle dimensioni totali che deve occupare il solido. Se le dimensioni delle celle non sono concordi con quelle massime del solido le prime verranno riadattate.

Successivamente testo la funzione

In [39]:
def door (Xlist,Ylist,Zlist,occupancy):
    """
    Funzione per generare famiglie di porte o finestre in 3D
    
    Args:
        Xlist (lista di reali): lista delle dimensioni sull'asse X di tutte le celle che compongono il solido
        Ylist (lista di reali): lista delle dimensioni sull'asse Y di tutte le celle che compongono il solido
        Zlist (lista di reali): lista delle dimensioni sull'asse Y di tutte le celle che compongono il solido
        occupancy (matrice tridimensionale booleana): matrice che rappresenta l'occupazione o meno di una cella da parte del "solido pieno"
        
    Returns:
        door0 (funzione): funzione annidata
    """
    def door0(dx,dy,dz):
        """
        Args:
            dx (numero reale): dimensione massima su x del solido
            dy (numero reale): dimensione massima su y del solido
            dz (numero reale): dimensione massima su z del solido
        
        Returns:
            structure (oggetto hpc): porta o finestra riprodotta
        """
        
        X = adaptListToDimension(Xlist,dx)
        Y = adaptListToDimension(Ylist,dy)
        Z = adaptListToDimension(Zlist,dz)
        
        occ = occupancy
        
        structure = STRUCT([CUBOID([0,0,0])])
        
        
        
        sumX = 0
        k = 0
        
        for plan in occ:
            
            page = STRUCT([CUBOID([0,0,0])])
            
            sumZ = 0
            i  = 0
            
            for row in plan :
                tmpY = []
                j=0
                
                allNegative = 1
                
                for val in row:
                
                    if val: 
                        tmpY.append(Y[j])
                        allNegative = 0
                        
                    else: tmpY.append(-Y[j])
                    
                    j+=1
                
                if not allNegative:
                    
                    qy = QUOTE(tmpY)
                    qz = QUOTE([Z[i]])
                    pxy = PROD([QUOTE([X[k]]),qy])
                    pyz = PROD([pxy,qz])
        
                    page = STRUCT([page,T([3])([sumZ]),pyz])
            
                sumZ += Z[i]
                i += 1
            
            structure = STRUCT([structure,T([1])([sumX]),page])
            
            sumX += X[k]
            k += 1
            
        
            
        return structure
    return door0
        
occ1 = [[1,1,1,1],[1,0,0,1],[1,1,1,1],[1,0,0,1],[1,1,1,1]]
occ2 = [[0,0,0,0],[0,1,1,0],[0,0,0,0],[0,1,1,0],[0,0,0,0]]
occ3 = [[1,1,1,1],[1,0,0,1],[1,1,1,1],[1,0,0,1],[1,1,1,1]]

#VIEW(door([.1,.1,.1],[.3,.3,.3,.1],[.1,.3,.4,.5,.2],[occ1,occ2,occ3])(1,2,3))
        

Estendo ulteriormente la funzione precedente introducendo nuovi "simboli" nella matrice che rappresenta l'occupazione del solido.
Invece di utilizzare semplici booleani (0 o 1) posso utilizzare n simboli o codici (es. n = 0,1,2,3...) per modellare il vuoto ed n-1 materiali differenti.
Viene eseguita la funzione precedente per ogni materiale che si intende modellare ed alla fine viene assemblato il tutto.
Viene introdotto un nuovo parametro ovvero la lista dei colori che si intendono utilizzare per rappresentare i diversi materiali. L'ordine con cui vengono applicati i colori è lo stesso di come sono stati scelti i simboli: ad esempio, essendo "0" il simbolo di default per rappresentare il vuoto, il primo colore della lista rappresenta il materiale indicato dal simbolo "1" nella matrice tridimensionale, il secondo colore rappresenta il materiale indicato dal simbolo "2" e così via.

Successivamente testo la funzione

In [40]:
def doorExtended (Xlist,Ylist,Zlist,occupancy,colorList):
    """
    Funzione per generare famiglie di porte o finestre in 3D
    
    Args:
        Xlist (lista di reali): lista delle dimensioni sull'asse X di tutte le celle che compongono il solido
        Ylist (lista di reali): lista delle dimensioni sull'asse Y di tutte le celle che compongono il solido
        Zlist (lista di reali): lista delle dimensioni sull'asse Y di tutte le celle che compongono il solido
        occupancy (matrice tridimensionale di interi): matrice che rappresenta l'occupazione o meno di una cella da parte del "solido pieno"
        colorList (lista di colori(liste di 3 numeri reali compresi tra 0 e 1)): sono i colori utilizzati nella modellazione
        
    Returns:
        door0 (funzione): funzione annidata
    """
    
    def door0(dx,dy,dz):
        """
        Args:
            dx (numero reale): dimensione massima su x del solido
            dy (numero reale): dimensione massima su y del solido
            dz (numero reale): dimensione massima su z del solido
        
        Returns:
            totalStructure (oggetto hpc): porta o finestra riprodotta
        """
        
        def door1(cod):
            """
            Args:
                cod (numero intero): è un codice che rappresenta un tipo di materiale
        
            Returns:
                structure (oggetto hpc): parte della struttura riprodotta composto da un singolo materiale, rappresentato da "cod"
            """
            
            X = adaptListToDimension(Xlist,dx)
            Y = adaptListToDimension(Ylist,dy)
            Z = adaptListToDimension(Zlist,dz)
        
            occ = occupancy

            structure = CUBOID([0,0,0])



            sumX = 0
            k = 0

            for plan in occ:

                page = CUBOID([0,0,0])

                sumZ = 0
                i  = 0

                for row in plan :
                    tmpY = []
                    j=0

                    allNegative = 1

                    for val in row:

                        if val == cod: 
                            tmpY.append(Y[j])
                            allNegative = 0

                        else: tmpY.append(-Y[j])

                        j+=1

                    if not allNegative:

                        qy = QUOTE(tmpY)
                        qz = QUOTE([Z[i]])
                        pxy = PROD([QUOTE([X[k]]),qy])
                        pyz = PROD([pxy,qz])

                        page = STRUCT([page,T([3])([sumZ]),pyz])

                    sumZ += Z[i]
                    i += 1

                structure = STRUCT([structure,T([1])([sumX]),page])

                sumX += X[k]
                k += 1

            return structure
        
        totalStructure = CUBOID([0,0,0])
        
        c = 0
        for val in colorList :
            partialStructure = COLOR(colorList[c])(door1(c+1))
            totalStructure = STRUCT([totalStructure,partialStructure])
            c += 1
        
        return totalStructure

    return door0
        
page1 = [[1,1,1,1],[1,0,0,1],[1,1,1,1],[1,0,0,1],[1,1,1,1]]
page2 = [[1,1,1,1],[1,2,2,1],[1,0,0,1],[1,2,2,1],[1,1,1,1]]
page3 = [[1,1,1,1],[1,0,0,1],[1,1,1,1],[1,0,0,1],[1,1,1,1]]

frameColor = [134.0/255,67.0/255,0.0/255]
glassColor = [195.0/255,248.0/255,255.0/255]

#VIEW(doorExtended([.1,.1,.1],[.3,.3,.3,.1],[.1,.3,.4,.5,.2],[page1,page2,page3],[frameColor,glassColor])(1,2,3))

N.B. Con quest' ultima estensione è possibile generare sia porte che finestre (in realtà anche molti altri tipi di solidi) utilizzando solo la funzione "door(...)(...)"

Definisco i colori che mi serviranno per la riproduzione dei modelli

In [41]:
woodColor = [94.0/255,41.0/255,0.0/255]
glassColor = [195.0/255,248.0/255,255.0/255]
steelColor = [192.0/255,192.0/255,192.0/255]
darkSteelColor = [96.0/255,96.0/255,96.0/255]
blackWoodColor = [53.0/255,53.0/255,53.0/255]
ornamentalRedColor = [128.0/255,18.0/255,18.0/255]
ornamentalBrownColor = [119.0/255,84.0/255,52.0/255]

## Riproduco "firstDoor"

#### Codici dei materiali:
- 0 = vuoto
- 1 = legno
- 2 = vetro

#### Modello

<img src="files/firstDoor.jpg" width="200">

#### Vista esterna

<img src="files/fdOutside.jpg" width="800">

#### Vista interna

<img src="files/fdInside.jpg" width="800">

#### Vista semi-profilo

<img src="files/fdSemiprofile.jpg" width="800">

Di seguito il codice utilizzato:

In [42]:

firstDoorX = [0.04,0.03,0.03,0.03,0.08,0.03,0.03,0.03,0.04]
firstDoorY = [0.1,0.02,0.005,0.15,0.25,0.5,0.25,0.15,0.005,0.02,0.1]
firstDoorZ = [0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.005,0.02,0.1]

page1 = [[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,1,1,1,1,1,1,1,1,1,1]]

page2 = [[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page3 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page4 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,1,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page5 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,2,2,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,2,2,1,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,2,2,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page6 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,1,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page7 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page8 = [[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page9 = [[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,1,0,0,0,0,0,0,0,1,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


pages = [page1,page2,page3,page4,page5,page6,page7,page8,page9]

dx = sumAllElements(firstDoorX)
dy = sumAllElements(firstDoorY)
dz = sumAllElements(firstDoorZ)

colorList = [woodColor,glassColor]

handle = COLOR(steelColor)(STRUCT([CUBOID([0.02,0.08,0.08]),
                 T([1,2,3])([0.02,0.02,0.02]),CUBOID([0.04,0.04,0.04]),
                 T([1,2])([0.04,0.02]),CUBOID([0.01,0.22,0.036])]))

cylinder = COLOR(steelColor)(CYLINDER([0.01,0.2])(50))

firstDoor = doorExtended(firstDoorX,firstDoorY,firstDoorZ,pages,colorList)(dx,dy,dz)

firstDoor = STRUCT([firstDoor,T([1,2,3])([0.3,0.14,1.4]),handle])
firstDoor = STRUCT([firstDoor,T([1,2,3])([0.3,1.43,0.3]),cylinder,T([3])([2.08]),cylinder])

VIEW(firstDoor)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000006C75C00> >

## Riproduco "secondDoor"

#### Codici dei materiali:
- 0 = vuoto
- 1 = legno nero
- 2 = acciaio
- 3 = acciaio scuro
- 4 = vetro

#### Modello

<img src="files/secondDoor.jpg" width="250">

#### Vista esterna

<img src="files/sdOutside.jpg" width="800">

#### Vista interna

<img src="files/sdInside.jpg" width="800">

#### Vista semi-profilo

<img src="files/sdSemiprofile.jpg" width="800">

Di seguito il codice utilizzato:

In [43]:

secondDoorX = [0.2,0.03,0.3,0.1,0.03]
secondDoorY = [0.1,0.5,0.2,0.5,0.1,0.3,0.1,0.5,0.2,0.5,0.1]
secondDoorZ = [0.1,0.3,0.4,0.05,0.4,0.05,0.4,0.05,0.4,0.05,0.4,0.4,0.1]

page1 = [[1,0,1,1,2,2,2,1,1,0,1],[1,0,1,1,2,2,2,1,1,0,1],[1,0,1,1,2,4,2,1,1,0,1],
         [1,0,1,1,2,3,2,1,1,0,1],[1,0,1,1,2,4,2,1,1,0,1],[1,0,1,1,2,3,2,1,1,0,1],
         [1,0,1,1,2,4,2,1,1,0,1],[1,0,1,1,2,3,2,1,1,0,1],[1,0,1,1,2,4,2,1,1,0,1],
         [1,0,1,1,2,3,2,1,1,0,1],[1,0,1,1,2,4,2,1,1,0,1],[1,0,1,1,2,2,2,1,1,0,1],
         [1,1,1,1,1,1,1,1,1,1,1]]

page2 = [[1,0,1,1,2,2,2,1,1,0,1],[1,0,1,1,2,2,2,1,1,0,1],[1,0,1,1,2,0,2,1,1,0,1],
         [1,0,1,1,2,0,2,1,1,0,1],[1,0,1,1,2,0,2,1,1,0,1],[1,0,1,1,2,0,2,1,1,0,1],
         [1,0,1,1,2,0,2,1,1,0,1],[1,0,1,1,2,0,2,1,1,0,1],[1,0,1,1,2,0,2,1,1,0,1],
         [1,0,1,1,2,0,2,1,1,0,1],[1,0,1,1,2,0,2,1,1,0,1],[1,0,1,1,2,2,2,1,1,0,1],
         [1,1,1,1,1,1,1,1,1,1,1]]

page3 = [[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,1,1,0,0,0,0,0,1,1,1]]

page4 = [[1,1,1,0,0,0,0,0,1,1,1],[1,4,1,0,0,0,0,0,1,4,1],[1,4,1,0,0,0,0,0,1,4,1],
         [1,3,1,0,0,0,0,0,1,3,1],[1,4,1,0,0,0,0,0,1,4,1],[1,3,1,0,0,0,0,0,1,3,1],
         [1,4,1,0,0,0,0,0,1,4,1],[1,3,1,0,0,0,0,0,1,3,1],[1,4,1,0,0,0,0,0,1,4,1],
         [1,3,1,0,0,0,0,0,1,3,1],[1,4,1,0,0,0,0,0,1,4,1],[1,4,1,0,0,0,0,0,1,4,1],
         [1,1,1,0,0,0,0,0,1,1,1]]

page5 = [[1,1,1,0,0,0,0,0,1,1,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],[1,0,1,0,0,0,0,0,1,0,1],
         [1,1,1,0,0,0,0,0,1,1,1]]

pages = [page1,page2,page3,page4,page5]

dx = sumAllElements(secondDoorX)
dy = sumAllElements(secondDoorY)
dz = sumAllElements(secondDoorZ)

colorList = [blackWoodColor,steelColor,darkSteelColor,glassColor]

weld = COLOR(steelColor)(STRUCT([CUBOID([0.1,0.04,0.04]),T(1)(0.1),CUBOID([0.04,0.1,0.04])]))
handle = COLOR(steelColor)(STRUCT([CYLINDER([0.04,2.2])(50),T([1,2,3])([-0.12,-0.12,0.3]),weld,T([3])([1.6]),weld]))

secondDoor = doorExtended(secondDoorX,secondDoorY,secondDoorZ,pages,colorList)(dx,dy,dz)

secondDoor = STRUCT([secondDoor,T([1,2,3])([0.35,1.05,0.4]),handle])

VIEW(secondDoor)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000006C74B10> >

## Riproduco "firstWindow"

#### Codici dei materiali:
- 0 = vuoto
- 1 = materiale marrone
- 2 = materiale rosso
- 3 = legno
- 4 = vetro

#### Modello

<img src="files/firstWindow.jpg" width="500">

#### Vista esterna

<img src="files/fwOutside.jpg" width="800">

#### Vista interna

<img src="files/fwInside.jpg" width="800">

#### Vista semi-profilo

<img src="files/fwSemiprofile.jpg" width="800">

Di seguito il codice utilizzato:

In [44]:
firstWindowX = [0.08,0.06,0.04,0.06,0.04,0.06,0.08,0.1]
firstWindowY = [0.02,0.1,0.08,0.15,0.03,0.8,0.03,0.15,0.08,0.1,0.02]
firstWindowZ = [0.12,0.15,0.15,0.03,2.2,0.03,0.15,0.12,0.15]

page1 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],
         [0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],
         [0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page2 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,0,0,0,0,0,3,3,0],
         [0,3,3,0,0,0,0,0,3,3,0],[0,3,3,0,0,0,0,0,3,3,0],[0,3,3,0,0,0,0,0,3,3,0],
         [0,3,3,0,0,0,0,0,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page3 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,0,3,0,3,0,3,3,0],
         [0,3,3,3,3,3,3,3,3,3,0],[0,3,3,0,3,0,3,0,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],
         [0,3,3,0,3,0,3,0,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page4 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,4,3,4,3,4,3,3,0],
         [0,3,3,3,3,3,3,3,3,3,0],[0,3,3,4,3,4,3,4,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],
         [0,3,3,4,3,4,3,4,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page5 = [[1,1,1,1,1,1,1,1,1,1,1],[0,1,2,2,2,2,2,2,2,1,0],[0,1,2,0,2,0,2,0,2,1,0],
         [0,1,2,2,2,2,2,2,2,1,0],[0,1,2,0,2,0,2,0,2,1,0],[0,1,2,2,2,2,2,2,2,1,0],
         [0,1,2,0,2,0,2,0,2,1,0],[0,1,2,2,2,2,2,2,2,1,0],[1,1,1,1,1,1,1,1,1,1,1]]

page6 = [[1,1,1,1,1,1,1,1,1,1,1],[0,1,2,2,2,2,2,2,2,1,0],[0,1,2,0,0,0,0,0,2,1,0],
         [0,1,2,0,0,0,0,0,2,1,0],[0,1,2,0,0,0,0,0,2,1,0],[0,1,2,0,0,0,0,0,2,1,0],
         [0,1,2,0,0,0,0,0,2,1,0],[0,1,2,2,2,2,2,2,2,1,0],[1,1,1,1,1,1,1,1,1,1,1]]

page7 = [[1,1,1,1,1,1,1,1,1,1,1],[0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],
         [0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],
         [0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],[1,1,1,1,1,1,1,1,1,1,1]]

page8 = [[1,1,1,1,1,1,1,1,1,1,1],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0]]

pages = [page1,page2,page3,page4,page5,page6,page7,page8]

dx = sumAllElements(firstWindowX)
dy = sumAllElements(firstWindowY)
dz = sumAllElements(firstWindowZ)

colorList = [ornamentalBrownColor,ornamentalRedColor,woodColor,glassColor]

firstWindow = doorExtended(firstWindowX,firstWindowY,firstWindowZ,pages,colorList)(dx,dy,dz)

VIEW(firstWindow)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000006D566F0> >

## Riproduco "secondWindow"

#### Codici dei materiali:
- 0 = vuoto
- 1 = legno nero
- 2 = vetro

#### Modello

<img src="files/secondWindow.jpg" width="250">

#### Vista esterna

<img src="files/swOutside.jpg" width="800">

#### Vista interna

<img src="files/swInside.jpg" width="800">

#### Vista semi-profilo

<img src="files/swSemiprofile.jpg" width="800">

Di seguito il codice utilizzato:

In [45]:
secondWindowX = [0.2,0.3,0.05,0.05,0.05,0.1]
secondWindowY = [0.2,0.08,1,0.1,1,0.08,0.2]
secondWindowZ = [1.5,0.15,0.06,1,0.1,1,0.4,1,0.08,0.2]

page1 = [[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],
         [1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],
         [1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,1,1,1,1,1,1]]

page2 = [[1,0,0,0,0,0,1],[1,1,1,1,1,1,1],[1,0,0,0,0,0,1],
         [1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],
         [1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,1,1,1,1,1,1]]

page3 = [[1,0,0,0,0,0,1],[1,1,1,1,1,1,1],[1,1,1,1,1,1,1],
         [1,1,0,1,0,1,1],[1,1,1,1,1,1,1],[1,1,0,1,0,1,1],
         [1,1,1,1,1,1,1],[1,1,0,1,0,1,1],[1,1,1,1,1,1,1],[1,1,1,1,1,1,1]]

page4 = [[1,0,0,0,0,0,1],[1,1,1,1,1,1,1],[1,1,1,1,1,1,1],
         [1,1,2,1,2,1,1],[1,1,1,1,1,1,1],[1,1,2,1,2,1,1],
         [1,1,1,1,1,1,1],[1,1,2,1,2,1,1],[1,1,1,1,1,1,1],[1,1,1,1,1,1,1]]

page5 = [[1,0,0,0,0,0,1],[1,1,1,1,1,1,1],[1,1,1,1,1,1,1],
         [1,1,0,1,0,1,1],[1,1,1,1,1,1,1],[1,1,0,1,0,1,1],
         [1,1,1,1,1,1,1],[1,1,0,1,0,1,1],[1,1,1,1,1,1,1],[1,1,1,1,1,1,1]]

page6 = [[1,0,0,0,0,0,1],[1,1,1,1,1,1,1],[1,0,0,0,0,0,1],
         [1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],
         [1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,0,0,0,0,0,1],[1,1,1,1,1,1,1]]



pages = [page1,page2,page3,page4,page5,page6]

dx = sumAllElements(secondWindowX)
dy = sumAllElements(secondWindowY)
dz = sumAllElements(secondWindowZ)

colorList = [blackWoodColor,glassColor]

secondWindow = doorExtended(secondWindowX,secondWindowY,secondWindowZ,pages,colorList)(dx,dy,dz)

VIEW(secondWindow)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000006D56600> >